In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/soil-classification-part-2/soil_competition-2025/sample_submission.csv
/kaggle/input/soil-classification-part-2/soil_competition-2025/train_labels.csv
/kaggle/input/soil-classification-part-2/soil_competition-2025/test_ids.csv
/kaggle/input/soil-classification-part-2/soil_competition-2025/test/465084323936570da664f0ca8dc90326.jpg
/kaggle/input/soil-classification-part-2/soil_competition-2025/test/1aa0b12029d35e778dba5bff1255c638.jpg
/kaggle/input/soil-classification-part-2/soil_competition-2025/test/6df2c3dcd4fb59298c7a73467ea72eeb.jpg
/kaggle/input/soil-classification-part-2/soil_competition-2025/test/107f25ebd87f581ea57c630a2dcdf50c.jpg
/kaggle/input/soil-classification-part-2/soil_competition-2025/test/dc35d58782615e4f9582c6b32c8b956e.jpg
/kaggle/input/soil-classification-part-2/soil_competition-2025/test/c7af21ff925c51adb526c487148bac6d.jpg
/kaggle/input/soil-classification-part-2/soil_competition-2025/test/e8bdb9805d455093ab4f9503cad8052b.jpg
/kaggle/input/soil-class

In [6]:
# pip install timm

import os, random, numpy as np, pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset, WeightedRandomSampler
from torchvision import datasets, transforms
from timm import create_model
from tqdm import tqdm

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths
SOIL_IMG_DIR = "/kaggle/input/soil-classification-part-2/soil_competition-2025/train"
SOIL_LABEL_CSV = "/kaggle/input/soil-classification-part-2/soil_competition-2025/train_labels.csv"

# Config
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
LR = 3e-4

# Transforms
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.4, 0.4, 0.4),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])
val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Dataset
class SoilDataset(Dataset):
    def __init__(self, df, img_dir, transform, label=1.0):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.label = label

    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.iloc[idx]['image_id'])
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        return image, torch.tensor(self.label, dtype=torch.float32)

class CIFARWrapper(Dataset):
    def __init__(self, dataset, transform, label=0.0):
        self.dataset = dataset
        self.transform = transform
        self.label = label

    def __len__(self): return len(self.dataset)

    def __getitem__(self, idx):
        image, _ = self.dataset[idx]
        image = self.transform(image)
        return image, torch.tensor(self.label, dtype=torch.float32)

# Load and split
soil_df = pd.read_csv(SOIL_LABEL_CSV)
train_df, val_df = train_test_split(soil_df, test_size=0.2, random_state=42)

# Soil datasets
soil_train = SoilDataset(train_df, SOIL_IMG_DIR, train_transform, 1.0)
soil_val = SoilDataset(val_df, SOIL_IMG_DIR, val_transform, 1.0)

# CIFAR10 negatives
cifar10_train = datasets.CIFAR10(".", train=True, download=True)
cifar10_val = datasets.CIFAR10(".", train=False, download=True)

# Wrap harder negatives
hard_classes = [2, 3, 4, 5, 6]  # bird, cat, deer, dog, frog
hard_train = [(x, y) for x, y in zip(cifar10_train.data, cifar10_train.targets) if y in hard_classes]
hard_val = [(x, y) for x, y in zip(cifar10_val.data, cifar10_val.targets) if y in hard_classes]

class NumpyImageDataset(Dataset):
    def __init__(self, data, transform, label=0.0):
        self.data = data
        self.transform = transform
        self.label = label

    def __len__(self): return len(self.data)

    def __getitem__(self, idx):
        image = Image.fromarray(self.data[idx][0] if isinstance(self.data[idx], tuple) else self.data[idx])
        image = self.transform(image)
        return image, torch.tensor(self.label, dtype=torch.float32)

non_soil_train = NumpyImageDataset(hard_train[:len(train_df)], train_transform)
non_soil_val = NumpyImageDataset(hard_val[:len(val_df)], val_transform)

# Combine datasets
train_dataset = ConcatDataset([soil_train, non_soil_train])
val_dataset = ConcatDataset([soil_val, non_soil_val])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

# Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.bce = nn.BCEWithLogitsLoss(reduction="none")

    def forward(self, inputs, targets):
        bce_loss = self.bce(inputs, targets)
        pt = torch.exp(-bce_loss)
        focal = self.alpha * (1 - pt) ** self.gamma * bce_loss
        return focal.mean()

# Model: ConvNeXt Base
model = create_model("convnext_base", pretrained=True, num_classes=1).to(DEVICE)
criterion = FocalLoss()
optimizer = optim.AdamW(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

# Mixed Precision
scaler = torch.cuda.amp.GradScaler()

# Train function
def train():
    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            images, labels = images.to(DEVICE), labels.to(DEVICE).unsqueeze(1)
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()
        scheduler.step()
        print(f"Epoch {epoch+1}: Train Loss = {total_loss/len(train_loader):.4f}")
        validate()

def validate():
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE).unsqueeze(1)
            outputs = torch.sigmoid(model(images))
            preds = (outputs > 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    print(f"Validation Accuracy: {100 * correct / total:.2f}%")

train()
torch.save(model.state_dict(), "convnext_soil_classifier.pth")
print("Model saved to convnext_soil_classifier.pth")


model.safetensors:   0%|          | 0.00/354M [00:00<?, ?B/s]

/tmp/ipykernel_35/981703507.py:136: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1:   0%|          | 0/62 [00:00<?, ?it/s]/tmp/ipykernel_35/981703507.py:146: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1: 100%|██████████| 62/62 [00:22<00:00,  2.80it/s]

Epoch 1: Train Loss = 0.0057


Validation Accuracy: 99.80%


Epoch 2: 100%|██████████| 62/62 [00:22<00:00,  2.71it/s]

...

Epoch 9: Train Loss = 0.0004


Validation Accuracy: 100.00%


Epoch 10: 100%|██████████| 62/62 [00:22<00:00,  2.77it/s]

Epoch 10: Train Loss = 0.0001


Validation Accuracy: 100.00%
Model saved to convnext_soil_classifier.pth


In [8]:
import os
import torch
import pandas as pd
from PIL import Image
from torchvision import transforms
from timm import create_model
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Configs
TEST_IMG_DIR = "/kaggle/input/soil-classification-part-2/soil_competition-2025/test"
MODEL_PATH = "/kaggle/working/convnext_soil_classifier.pth"
BATCH_SIZE = 32
IMG_SIZE = 224
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transforms (same as validation)
test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Dataset for test
class TestDataset(Dataset):
    def __init__(self, img_dir, transform):
        self.img_dir = img_dir
        self.transform = transform
        self.image_ids = sorted(os.listdir(img_dir))

    def __len__(self): return len(self.image_ids)

    def __getitem__(self, idx):
        img_name = self.image_ids[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        return image, img_name

# Load test data
test_dataset = TestDataset(TEST_IMG_DIR, test_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Load model
model = create_model("convnext_base", pretrained=False, num_classes=1)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# Predict
results = []
with torch.no_grad():
    for images, filenames in tqdm(test_loader, desc="Predicting"):
        images = images.to(DEVICE)
        outputs = torch.sigmoid(model(images)).squeeze(1)
        preds = (outputs > 0.5).long().cpu().numpy()
        results.extend(zip(filenames, preds))

# Create submission
submission_df = pd.DataFrame(results, columns=["image_id", "label"])
submission_df.to_csv("submission.csv", index=False)
print("Saved predictions to submission.csv")


Predicting: 100%|██████████| 31/31 [00:13<00:00,  2.36it/s]

Saved predictions to submission.csv
